# PASO 1 — Librerías

In [2]:
import os
import pandas as pd

# 1. Borrar la carpeta si ya existe (por si corres la celda varias veces)
!rm -rf Comparative_analysis_Competitive_strategies

# 2. Clonar tu repositorio de GitHub
!git clone https://github.com/andresperez86/Comparative_analysis_Competitive_strategies.git

# 3. Entrar a la carpeta del proyecto
%cd Comparative_analysis_Competitive_strategies

# 4. Verificar que la carpeta 'data' está ahí
if os.path.exists("data"):
    print("✅ Carpeta de datos cargada correctamente.")
    print("Archivos disponibles:", os.listdir("data"))

# 5. Ejemplo de carga de un archivo (EIA.xlsx)
# Ahora la ruta es simplemente 'data/nombre_archivo.xlsx'
df_eia = pd.read_excel("data/EIA.xlsx")
print("\nVista previa de EIA.xlsx:")
print(df_eia.head())

Cloning into 'Comparative_analysis_Competitive_strategies'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 29 (delta 4), reused 25 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 12.54 MiB | 27.20 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/Comparative_analysis_Competitive_strategies/Comparative_analysis_Competitive_strategies
✅ Carpeta de datos cargada correctamente.
Archivos disponibles: ['VTECCI.xlsx', 'OMEGA.xlsx', 'EIA.xlsx', 'Sabana.xlsx', 'pascual bravo.xlsx', 'KRATOS.xlsx', 'HELLMEC.xlsx']

Vista previa de EIA.xlsx:
                    timestamp  car_velocity  car_trip  car_velocity_gps  \
0  2025-11-07T20:18:02.837767           0.0       0.0              0.00   
1  2025-11-07T20:18:03.843626           0.0       0.0             16.65   
2  2025-11-07T20:18:04.845043           0.0       0.0             31.45   
3  2025-11-07T20:18:05.847000     

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

PASO 2 — Cargar todos los equipos automáticamente

In [4]:
import os
print(os.getcwd())

/content/Comparative_analysis_Competitive_strategies/Comparative_analysis_Competitive_strategies


In [5]:
files = {
    "EIA": "/content/Comparative_analysis_Competitive_strategies/data/EIA.xlsx",
    "HELLMEC": "/content/Comparative_analysis_Competitive_strategies/data/HELLMEC.xlsx",
    "KRATOS": "/content/Comparative_analysis_Competitive_strategies/data/OMEGA.xlsx",
    "PASCUAL": "/content/Comparative_analysis_Competitive_strategies/data/pascual bravo.xlsx",
    "SABANA": "/content/Comparative_analysis_Competitive_strategies/data/Sabana.xlsx",
    "VTECCI": "/content/Comparative_analysis_Competitive_strategies/data/VTECCI.xlsx",
}

dfs = {}

#file_path = os.path.join("..", "/content/data", "EIA.xlsx")

for team, path in files.items():
  print(f"Loading data for {team} from {path}")
  df = pd.read_excel(path)
  df["team"] = team
  dfs[team] = df

Loading data for EIA from /content/Comparative_analysis_Competitive_strategies/data/EIA.xlsx
Loading data for HELLMEC from /content/Comparative_analysis_Competitive_strategies/data/HELLMEC.xlsx
Loading data for KRATOS from /content/Comparative_analysis_Competitive_strategies/data/OMEGA.xlsx
Loading data for PASCUAL from /content/Comparative_analysis_Competitive_strategies/data/pascual bravo.xlsx
Loading data for SABANA from /content/Comparative_analysis_Competitive_strategies/data/Sabana.xlsx
Loading data for VTECCI from /content/Comparative_analysis_Competitive_strategies/data/VTECCI.xlsx


PASO 3 — Limpieza y estandarización

In [6]:
def clean_data(df):
    df = df.copy()
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df = df.sort_values("timestamp")

    numeric_cols = ["car_velocity", "car_voltage", "car_current", "power"]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df = df.dropna(subset=["timestamp"])
    df["dt"] = df["timestamp"].diff().dt.total_seconds()
    df["dt"] = df["dt"].fillna(0)

    return df

for team in dfs:
    dfs[team] = clean_data(dfs[team])

PASO 4 — Cálculo de energía y eficiencia

In [7]:
def compute_metrics(df):
    df = df.copy()

    # Energía en Wh
    df["energy_Wh"] = (df["power"] * df["dt"]) / 3600

    total_energy = df["energy_Wh"].sum()

    # Distancia
    if "car_trip" in df.columns and df["car_trip"].max() > 0:
        distance = df["car_trip"].max()
    else:
        df["distance_m"] = df["car_velocity"] * df["dt"]
        distance = df["distance_m"].sum() / 1000

    efficiency = total_energy / distance if distance > 0 else np.nan

    return {
        "Energy_Wh": total_energy,
        "Distance_km": distance,
        "Efficiency_Wh_per_km": efficiency,
        "Mean_Speed": df["car_velocity"].mean(),
        "Speed_std": df["car_velocity"].std(),
        "Power_mean": df["power"].mean(),
        "Power_p95": df["power"].quantile(0.95),
        "Coasting_ratio": np.mean(np.abs(df["power"]) < 5),
        "Regen_ratio": np.mean(df["power"] < 0)
    }

summary = []

for team, df in dfs.items():
    metrics = compute_metrics(df)
    metrics["Team"] = team
    summary.append(metrics)

summary_df = pd.DataFrame(summary)
summary_df = summary_df.sort_values("Efficiency_Wh_per_km")
summary_df

,Energy_Wh,Distance_km,Efficiency_Wh_per_km,Mean_Speed,Speed_std,Power_mean,Power_p95,Coasting_ratio,Regen_ratio,Team
3,144.736338,19.048840,7.598171,7.043440,13.131078,81.428249,633.722710,0.473996,0.327151,PASCUAL
0,252.268611,19.118094,13.195281,10.988073,16.419881,144.697256,868.951920,0.588401,0.249805,EIA
2,289.586662,20.612289,14.049224,4.396387,11.144329,62.506562,521.634168,0.802406,0.543493,KRATOS
1,134.346895,6.322658,21.248482,1.747278,29.245639,36.244367,139.277783,0.616202,0.205711,HELLMEC
4,192.156354,0.000000,NaN,0.000000,0.000000,43.034747,413.208369,0.815331,0.351971,SABANA
5,233.571604,0.000000,NaN,0.000000,0.000000,78.398144,589.189507,0.657597,0.068814,VTECCI


📌 Esta tabla es la principal del paper.

Guardar:

In [10]:
summary_df.to_csv("/content/Comparative_analysis_Competitive_strategies/outputs/team_summary.csv", index=False)#/content/Comparative_analysis_Competitive_strategies/outputs